# Project: Cotton Plant Disease Prediction & Get Cure App

In [ ]:
# import libraries
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# for accuracy and loss graph
import matplotlib.pyplot as plt

In [ ]:
keras.__version__



In [ ]:
train_data_path = '/content/drive/MyDrive/Cotton Disease/train'
validation_data_path = '/content/drive/MyDrive/Cotton Disease/val'

In [ ]:
## Show augmented images
def plot_images(image_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20, 20))
  axes = axes.flatten()
  for img, ax in zip(image_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

In [ ]:
# this is the augmentation configuration we will use for training
# It generate more images using below parameters

training_datagen = ImageDataGenerator(rescale = 1./255,
                                      rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range=0.2,
                                      shear_range = 0.20,
                                      zoom_range = 0.20,
                                      horizontal_flip = True,
                                      fill_mode = 'nearest')

# This is a generator that will read pictures found in at train_data_path, and indefinitely generate batches
# of augmented image data

In [ ]:
training_data = training_datagen.flow_from_directory(train_data_path,     # This is a target directory
                                      target_size = (150, 150),  # all image resized to 150x150
                                      batch_size = 32, 
                                      class_mode = 'binary') # Since we use binary_crossentropy loss, we need binary labels 
 
training_data.class_indices                                        

In [ ]:
# this is the augmentation configuration we will use for validation
# Only rescalling
valid_design = ImageDataGenerator(rescale = 1./255)

# This is a similar generator , for validation data
valid_data = valid_design.flow_from_directory(validation_data_path,
                                               target_size = (150, 150),
                                               batch_size= 32,
                                               class_mode= 'binary')



In [ ]:
# Showing augmented images
images = [training_data[0][0][0] for i in range(5)]
plot_images(images)

In [ ]:
model_path = '/content/drive/MyDrive/Cotton Disease/decease_prediction.h5'
checkpoint = ModelCheckpoint(model_path, monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max')
callback_list = [checkpoint]

# Building cnn model


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Conv2D

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, strides = (1,1), kernel_size  = 3, padding = 'valid', activation = 'relu', input_shape = [150, 150, 3] ))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64, kernel_size = 3))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 128, kernel_size = 3))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 256, kernel_size = 3))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.5))

model.add(Flatten() )
model.add(Dense( units = 128, activation= 'relu' ))
model.add(Dropout(0.1))

model.add(Dense( units = 256, activation= 'relu' ))
model.add(Dropout(0.25))

model.add(Dense( units = 4, activation= 'softmax' ))

In [ ]:
# Compile cnn model
model.compile(optimizer = Adam(lr = 0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']  )

In [ ]:
model.summary()

In [ ]:
history = model.fit(training_data, epochs = 500, verbose = 1, validation_data = valid_data, callbacks = callback_list )  

In [ ]:
# model_path2 = 'content/drive/MyDrive/'
# model_path

In [ ]:
# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend( ['train', 'test'], loc = 'upper left' )
plt.show()

In [ ]:
# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc = 'upper left')